# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import cartopy.crs as ccrs
import geoviews as gv

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,margaret river,-33.9500,115.0667,14.67,68,30,8.52,AU,1689890978
1,1,castillos,-34.1667,-53.8333,13.11,88,99,4.14,UY,1689890978
2,2,iqaluit,63.7506,-68.5145,4.85,100,100,3.09,CA,1689890978
3,3,adamstown,-25.0660,-130.1015,22.30,86,100,17.86,PN,1689890978
4,4,longyearbyen,78.2186,15.6401,10.91,71,20,8.23,SJ,1689890978


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
weather_filtered_df = city_data_df.loc[(city_data_df["Max Temp"] < 27)
                                       & (city_data_df["Max Temp"] > 21) 
                                       & (city_data_df["Cloudiness"] < 3)
                                       & (city_data_df["Wind Speed"] < 4.5)
                                      ]

# Drop any rows with null values
weather_filtered_df.dropna()

# Display sample data
weather_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
46,46,remire-montjoly,4.9167,-52.2667,25.02,88,0,1.03,GF,1689891113
63,63,sannicandro garganico,41.8384,15.5654,26.00,65,0,2.03,IT,1689891210
104,104,aibak,36.2647,68.0155,24.95,32,0,2.27,AF,1689891215
151,151,al bayda',32.7627,21.7551,22.27,72,0,1.53,LY,1689891221
164,164,palpa,-14.5336,-75.1856,25.44,42,0,3.19,PE,1689891222
169,169,mersa matruh,31.3525,27.2453,26.30,66,0,2.17,EG,1689891222
207,207,torbat-e heydariyeh,35.2740,59.2195,21.93,36,0,3.82,IR,1689891226
229,229,bingol,38.8847,40.4939,21.89,19,0,1.65,TR,1689891229
286,286,sason,38.3364,41.4130,26.10,14,0,2.07,TR,1689891234
309,309,dauphin,51.1494,-100.0502,26.08,53,0,4.12,CA,1689891237


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
46,remire-montjoly,GF,4.9167,-52.2667,88,
63,sannicandro garganico,IT,41.8384,15.5654,65,
104,aibak,AF,36.2647,68.0155,32,
151,al bayda',LY,32.7627,21.7551,72,
164,palpa,PE,-14.5336,-75.1856,42,
169,mersa matruh,EG,31.3525,27.2453,66,
207,torbat-e heydariyeh,IR,35.2740,59.2195,36,
229,bingol,TR,38.8847,40.4939,19,
286,sason,TR,38.3364,41.4130,14,
309,dauphin,CA,51.1494,-100.0502,53,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel", "apiKey": geoapify_key}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json() 
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
remire-montjoly - nearest hotel: Complexe Belova
sannicandro garganico - nearest hotel: No hotel found
aibak - nearest hotel: No hotel found
al bayda' - nearest hotel: فندق الهديل
palpa - nearest hotel: Alubias
mersa matruh - nearest hotel: صقور
torbat-e heydariyeh - nearest hotel: هتل رستوران سنتی لاری
bingol - nearest hotel: Grand Hotel Berti
sason - nearest hotel: Sason Öğretmenevi
dauphin - nearest hotel: No hotel found
urumqi - nearest hotel: 百时快捷酒店红山店
bela vista - nearest hotel: Bella Vista Palace
princeville - nearest hotel: No hotel found
kyzylorda - nearest hotel: Нур
gudul - nearest hotel: No hotel found
paphos - nearest hotel: Mayfair Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
46,remire-montjoly,GF,4.9167,-52.2667,88,Complexe Belova
63,sannicandro garganico,IT,41.8384,15.5654,65,No hotel found
104,aibak,AF,36.2647,68.0155,32,No hotel found
151,al bayda',LY,32.7627,21.7551,72,فندق الهديل
164,palpa,PE,-14.5336,-75.1856,42,Alubias
169,mersa matruh,EG,31.3525,27.2453,66,صقور
207,torbat-e heydariyeh,IR,35.2740,59.2195,36,هتل رستوران سنتی لاری
229,bingol,TR,38.8847,40.4939,19,Grand Hotel Berti
286,sason,TR,38.3364,41.4130,14,Sason Öğretmenevi
309,dauphin,CA,51.1494,-100.0502,53,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)